## Practice Project

### Imports & Data Retrieval

In [54]:
import numpy as np
from pylab import mpl, plt
plt.style.use('seaborn-v0_8')
import tpqoa
api = tpqoa.tpqoa('oanda.cfg')

In [55]:
instrument = 'EUR_USD'
start = '2024-04-07'
end = '2024-04-14'
granularity = 'M5'
price = 'M'

In [56]:
data = api.get_history(instrument, start, end,
                       granularity, price)

In [57]:
data = data[['c']]

In [58]:
data['return'] = np.log(data['c'] / data['c'].shift(1))

In [59]:
data['direction'] = np.where(data['return'] > 0, 1, -1)

In [72]:
data

,c,return,direction,lag_1,lag_2,lag_3,prediction
time,,,,,,,
2024-04-07 21:20:00,1.08380,-0.000018,-1,-0.000037,0.000129,-0.000101,0.001675
2024-04-07 21:25:00,1.08378,-0.000018,-1,-0.000018,-0.000037,0.000129,-0.124629
2024-04-07 21:30:00,1.08376,-0.000018,-1,-0.000018,-0.000018,-0.000037,-0.012935
2024-04-07 21:35:00,1.08372,-0.000037,-1,-0.000018,-0.000018,-0.000018,-0.042424
2024-04-07 21:40:00,1.08370,-0.000018,-1,-0.000037,-0.000018,-0.000018,-0.012935
...,...,...,...,...,...,...,...
2024-04-12 20:35:00,1.06421,0.000056,1,-0.000038,-0.000009,-0.000019,-0.001965
2024-04-12 20:40:00,1.06425,0.000038,1,0.000056,-0.000038,-0.000009,-0.111111
2024-04-12 20:45:00,1.06426,0.000009,1,0.000038,0.000056,-0.000038,-0.035343


In [60]:
data.to_csv('check.csv')

In [61]:
lags = 3

In [62]:
cols = []
for lag in range(1, lags + 1):
    col = f'lag_{lag}'
    data[col] = data['return'].shift(lag)
    cols.append(col)
data.dropna(inplace=True)

### Backtesting Imports

In [16]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
regr_1 = DecisionTreeRegressor(max_depth=5)

In [ ]:
regr_2 = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=5), n_estimators=300, random_state=None
)

In [ ]:
regr_1.fit(data[cols], np.sign(data['return']))

In [ ]:
regr_2.fit(data[cols], np.sign(data['return']))

In [ ]:
return_1 = regr_1.predict(data[cols])

In [ ]:
data['prediction'] = regr_2.predict(data[cols])

In [ ]:
data['prediction'].value_counts()

In [ ]:
data['strategy'] = data['prediction'] * data['return']

In [ ]:
data[['return', 'strategy']].cumsum().apply(np.exp).plot(
    figsize=(10, 6));

### Round 2 -- Simplifying it.
* fit(data[cols], data['returns'])
* (input data, the target vector)

In [68]:
base_model = DecisionTreeRegressor(max_depth=4)
base_model.fit(data[cols], data['direction']) #Use The Columns to predict returns

DecisionTreeRegressor(max_depth=4)

In [69]:
boost_reg = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth=4), n_estimators=300, random_state=42)
boost_reg.fit(data[cols], data['direction'])

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=4),
                  n_estimators=300, random_state=42)

In [70]:
data['prediction'] = boost_reg.predict(data[cols])

In [71]:
data['prediction']

time
2024-04-07 21:20:00    0.001675
2024-04-07 21:25:00   -0.124629
2024-04-07 21:30:00   -0.012935
2024-04-07 21:35:00   -0.042424
2024-04-07 21:40:00   -0.012935
                         ...   
2024-04-12 20:35:00   -0.001965
2024-04-12 20:40:00   -0.111111
2024-04-12 20:45:00   -0.035343
2024-04-12 20:50:00   -0.110459
2024-04-12 20:55:00   -0.001541
Name: prediction, Length: 1436, dtype: float64